In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time

In [2]:
sns.set()
pd.set_option('display.max_columns', None)

Мои приложения:  
https://vk.com/apps?act=manage

Получения токена во вКонтакте:  
https://oauth.vk.com/authorize?client_id=НОМЕР_МОЕГО_ПРИЛОЖЕНИЯ&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.52

In [15]:
token = ''

Методы API vk: https://dev.vk.com/method

Получим данные о первом пользователе vk

In [8]:
url = 'https://api.vk.com/method/users.get'
params = {
    'user_ids': '1',
    'access_token': token,
    'v': '5.131'
}

In [9]:
response = requests.get(url, params=params)

In [10]:
response.json()

{'response': [{'id': 1,
   'first_name': 'Павел',
   'last_name': 'Дуров',
   'can_access_closed': True,
   'is_closed': False}]}

10 самых популярных групп по теме

In [14]:
url = 'https://api.vk.com/method/groups.search'
params = {
    'q': 'judo',
    'access_token': token,
    'v': '5.131',
    'sort': 2,
    'count': 20
}

In [15]:
response = requests.get(url, params=params)

In [17]:
df = pd.DataFrame(response.json()['response']['items'])

Количество постов в группах

In [18]:
group_ids = ''
for id in df['id']:
    group_ids += str(id) + ', '

In [19]:
group_ids

'101721019, 30989892, 63767190, 34953427, 77865186, 130715458, 30706224, 169724294, 114536759, 141732800, 159415063, 38895380, 141197284, 131025153, 85077325, 85120233, 138562437, 71171542, 215580968, 137349272, '

In [20]:
url = 'https://api.vk.com/method/groups.getById'
params = {
    'access_token': token,
    'v': '5.131',
    'group_ids': group_ids,
    'fields': 'members_count, description'
}
response = requests.get(url, params=params)

In [21]:
df = pd.DataFrame(response.json()['response']).sort_values('members_count', ascending=False)

In [22]:
df

,id,description,members_count,name,screen_name,is_closed,type,photo_50,photo_100,photo_200
11,38895380,,207244,Дзюдо | Judo,zal_judo,0,page,https://sun9-26.userapi.com/s/v1/if1/Kdc4ft5Xj...,https://sun9-26.userapi.com/s/v1/if1/UmfSeFHSb...,https://sun9-26.userapi.com/s/v1/if1/ninSBnNjL...
3,34953427,"Рассказываем обо всем, что вам стоит знать о м...",102338,Judo News Online,judonews,0,page,https://sun9-25.userapi.com/s/v1/if1/j7Wdc3tYO...,https://sun9-25.userapi.com/s/v1/if1/NCcIYKauy...,https://sun9-25.userapi.com/s/v1/if1/IPX3UJA2g...
1,30989892,🌍First JUDO NEWS\n🥋официальный дистрибьютер IJ...,35320,First JUDO NEWS,essimostore_judo,0,group,https://sun9-18.userapi.com/s/v1/if1/i4SKXtFmS...,https://sun9-18.userapi.com/s/v1/if1/P-jKXfiM9...,https://sun9-18.userapi.com/s/v1/if1/cWkJxsSmH...
0,101721019,,19971,Дзюдо: медиатека,judo_media,0,group,https://sun9-56.userapi.com/s/v1/if1/ws2hpIJyH...,https://sun9-56.userapi.com/s/v1/if1/2b6q5NQN0...,https://sun9-56.userapi.com/s/v1/if1/2wtVOvPNn...
7,169724294,Все самое лучшее из мира дзюдо,15006,Дзюдо/Judo,club169724294,0,page,https://sun9-66.userapi.com/s/v1/if1/BqmtV6iSo...,https://sun9-66.userapi.com/s/v1/if1/2xJqDzOxI...,https://sun9-66.userapi.com/s/v1/if1/sDqzCMAYd...
2,63767190,Страница о грузинском дзюдо.\nНовости\nинтерес...,12934,Georgian Judo(Дзюдо Грузии),georgian_judo,0,page,https://sun9-47.userapi.com/s/v1/ig2/G5dyvor0A...,https://sun9-47.userapi.com/s/v1/ig2/9gQchJySm...,https://sun9-47.userapi.com/s/v1/ig2/A0BQ6DLbA...
17,71171542,Все мы живем в одном удивительном мире - мире ...,5837,#judo,judotag,0,page,https://sun9-59.userapi.com/s/v1/if1/N-VObc8bt...,https://sun9-59.userapi.com/s/v1/if1/5gk1lcmU-...,https://sun9-59.userapi.com/s/v1/if1/DydNOh9eq...
10,159415063,Данное сообщество посвященно судейству Дзюдо.,4437,Судейство Дзюдо | Judo refereeing,refereeru,0,page,https://sun9-10.userapi.com/s/v1/if1/diIy9_oxn...,https://sun9-10.userapi.com/s/v1/if1/-cZSxf0ta...,https://sun9-10.userapi.com/s/v1/if1/En55lKyl7...
14,85077325,Следите за нами в ВКонтакте \n в YouTube https...,1694,"Professional judo club ""OLYMP"" | Дзюдо | Иваново",olymp_ivanovo,0,group,https://sun9-77.userapi.com/s/v1/ig2/KZ2KlhJ7o...,https://sun9-77.userapi.com/s/v1/ig2/Ou8OyCUqj...,https://sun9-77.userapi.com/s/v1/ig2/hQH2NixrP...
5,130715458,"кимоно.нашивки,спортивные товары,сувениры.",1670,OMEGA JUDO SAMBO,omegajudosambo,0,page,https://sun9-45.userapi.com/s/v1/ig2/eVekeklkB...,https://sun9-45.userapi.com/s/v1/ig2/SnuALz1Zd...,https://sun9-45.userapi.com/s/v1/ig2/8jfEge0PL...


Ищем количество подписчиков сообществ  
Можно за раз выгрузить не более 1000 человек. Делаем цикл с offsetом.

In [16]:
url = 'https://api.vk.com/method/groups.getMembers'
offset = 0
max_count = 1000
df = pd.DataFrame()

while True:
    params = {
        'group_id': 101721019,
        'access_token': token,
        'v': '5.131',
        'fields': 'city',
        'offset': offset  # смещение после которого забираем данные
    }
    response = requests.get(url, params=params)
    result = response.json()
    
    if len(result['response']['items']) != 0:
        df = pd.concat([df, pd.DataFrame(result['response']['items'])])
    else:
        break
    offset += max_count
    time.sleep(0.6)  # api вКонтакте не позволяет чаще 0,5с обращаться

print('Количество участников:', len(df))

Количество участников: 17850


In [32]:
cities = df.city.dropna().apply(lambda x: x['title'])

In [33]:
cities.value_counts()

Москва             997
Санкт-Петербург    770
Челябинск          209
Алматы             188
Пермь              166
                  ... 
Palermo              1
Düsseldorf           1
Таскала              1
Мосальск             1
Томское              1
Name: city, Length: 1356, dtype: int64